In [37]:
# Import
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import  Activation,Dense, Conv2D, MaxPooling2D , Flatten, AveragePooling2D
from keras import optimizers
from keras.utils import to_categorical
from tensorflow.keras.losses import sparse_categorical_crossentropy
from sklearn.metrics import f1_score


# Import CNN from scratch
from cnn import *
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [9]:
# Load Dataset
(X,y),(X_test,y_test) = datasets.cifar10.load_data()
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,train_size=0.8)

# Normalize 
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

categories = ["airplanes", "cars", "birds", "cats", "deer", "dogs", "frogs", "horses", "ships","trucks"]

In [2]:
# Model 1: Baseline

model = Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy','F1Score']
)

model.load_weights("./training_result/model1.weights.h5")


/home/hnf/.pyenv/versions/3.12.9/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748482100.069393  205760 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1138 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/hnf/.pyenv/versions/3.12.9/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Feedforward Model from scratch
# Using Baseline Model (Model 1)
conv1_W, conv1_b = model.layers[0].get_weights()
conv2_W, conv2_b = model.layers[1].get_weights()  
conv3_W, conv3_b = model.layers[3].get_weights()
conv4_W, conv4_b = model.layers[4].get_weights()
conv5_W, conv5_b = model.layers[6].get_weights()
conv6_W, conv6_b = model.layers[7].get_weights()
dense1_W, dense1_b = model.layers[10].get_weights()
dense2_W, dense2_b = model.layers[11].get_weights()

conv1 = Conv2DLayer(kernel=conv1_W, bias=conv1_b, activation='relu')
conv2 = Conv2DLayer(kernel=conv2_W, bias=conv2_b, activation='relu')

conv3 = Conv2DLayer(kernel=conv3_W, bias=conv3_b, activation='relu')
conv4 = Conv2DLayer(kernel=conv4_W, bias=conv4_b, activation='relu')

conv5 = Conv2DLayer(kernel=conv5_W, bias=conv5_b, activation='relu')
conv6 = Conv2DLayer(kernel=conv6_W, bias=conv6_b, activation='relu')

dense1 = DenseLayer(W=dense1_W, b=dense1_b, activation='relu')
dense2 = DenseLayer(W=dense2_W, b=dense2_b, activation='softmax')

pool1 = Pooling2DLayer(pool_size=(2, 2), strides=(2, 2), type='max')
pool2 = Pooling2DLayer(pool_size=(2, 2), strides=(2, 2), type='max')
pool3 = Pooling2DLayer(pool_size=(2, 2), strides=(2, 2), type='max')

flatten = FlattenLayer()


In [12]:
# Predict function
def predict_single(x):
    x = conv1.forward(x)
    x = conv2.forward(x)
    x = pool1.forward(x)
    x = conv3.forward(x)
    x = conv4.forward(x)
    x = pool2.forward(x)
    x = conv5.forward(x)
    x = conv6.forward(x)
    x = pool3.forward(x)
    x = flatten.forward(x)
    x = dense1.forward(x)
    output = dense2.forward(x)
    return output

def predict_thread(X):
    results = []
    with ThreadPoolExecutor() as executor:
        for res in tqdm(executor.map(predict_single, X), total=len(X), desc="Predicting"):
            results.append(res)
    return results

In [10]:
library_results = model.predict(X_test)

2025-05-29 08:30:49.776953: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2025-05-29 08:30:49.840391: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
I0000 00:00:1748482249.993475  206034 service.cc:152] XLA service 0x71ece00036b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748482249.993504  206034 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-05-29 08:30:50.002508: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748482250.066390  206034 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-05-29 08:30:50.546909: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning

 51/313 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

I0000 00:00:1748482251.907721  206034 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step


In [13]:
manual_results = predict_thread(X_test)

Predicting: 100%|██████████| 10000/10000 [04:27<00:00, 37.36it/s]


In [40]:
# Convert to class

pred_manual = []
pred_library = []
for i in range(len(X_test)):
    manual = np.argmax(manual_results[i])
    library = np.argmax(library_results[i])
    pred_manual.append([int(manual)])
    pred_library.append([int(library)])

In [41]:
f1 = f1_score(y_test, np.array(pred_library).flatten(), average='macro')
print("F1 Score (macro):", f1)

F1 Score (macro): 0.7612228830405996


In [42]:
f1 = f1_score(y_test, np.array(pred_manual).flatten(), average='macro')
print("F1 Score (macro):", f1)

F1 Score (macro): 0.7613187146552625


In [48]:
diff = 0
for i in range(len(X_test)):
    if(y_test[i][0]!=pred_library[i][0]):
        print("Diff at index:",i)
        print("Predicted(library):",categories[pred_library[i][0]])
        print("True Val:",categories[y_test[i][0]])
        diff +=1

print("Total diff:",diff,"out of",len(X_test))


Diff at index: 7
Predicted(library): deer
True Val: frogs
Diff at index: 9
Predicted(library): trucks
True Val: cars
Diff at index: 10
Predicted(library): cats
True Val: airplanes
Diff at index: 21
Predicted(library): birds
True Val: airplanes
Diff at index: 25
Predicted(library): cats
True Val: birds
Diff at index: 26
Predicted(library): dogs
True Val: deer
Diff at index: 32
Predicted(library): birds
True Val: deer
Diff at index: 33
Predicted(library): cats
True Val: dogs
Diff at index: 35
Predicted(library): cars
True Val: birds
Diff at index: 46
Predicted(library): dogs
True Val: cats
Diff at index: 47
Predicted(library): cats
True Val: trucks
Diff at index: 52
Predicted(library): horses
True Val: airplanes
Diff at index: 57
Predicted(library): cats
True Val: horses
Diff at index: 58
Predicted(library): cats
True Val: deer
Diff at index: 59
Predicted(library): cats
True Val: frogs
Diff at index: 61
Predicted(library): dogs
True Val: cats
Diff at index: 66
Predicted(library): ships
T

In [49]:
diff = 0
for i in range(len(X_test)):
    if(y_test[i][0]!=pred_manual[i][0]):
        print("Diff at index:",i)
        print("Predicted(manual):",categories[pred_manual[i][0]])
        print("True Val:",categories[y_test[i][0]])
        diff +=1

print("Total diff:",diff,"out of",len(X_test))

Diff at index: 7
Predicted(manual): deer
True Val: frogs
Diff at index: 9
Predicted(manual): trucks
True Val: cars
Diff at index: 10
Predicted(manual): cats
True Val: airplanes
Diff at index: 21
Predicted(manual): birds
True Val: airplanes
Diff at index: 25
Predicted(manual): cats
True Val: birds
Diff at index: 26
Predicted(manual): dogs
True Val: deer
Diff at index: 32
Predicted(manual): birds
True Val: deer
Diff at index: 33
Predicted(manual): cats
True Val: dogs
Diff at index: 35
Predicted(manual): cars
True Val: birds
Diff at index: 46
Predicted(manual): dogs
True Val: cats
Diff at index: 47
Predicted(manual): cats
True Val: trucks
Diff at index: 52
Predicted(manual): horses
True Val: airplanes
Diff at index: 57
Predicted(manual): cats
True Val: horses
Diff at index: 58
Predicted(manual): cats
True Val: deer
Diff at index: 59
Predicted(manual): cats
True Val: frogs
Diff at index: 61
Predicted(manual): dogs
True Val: cats
Diff at index: 66
Predicted(manual): ships
True Val: cars
Dif